In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

path = 'ncaam-march-mania-2021/MDataFiles_Stage2/'

In [7]:
mapping = {1:'Rank',
           2:'TeamName',
           3:'Division',
           4:'Record',
           5:'AdjEM',
           6:'AdjO',
           7:'AdjORank',
           8:'AdjD',
           9:'AdjDRank',
           10:'AdjT',
           11:'AdjTRank',
           12:'Luck',
           13:'LuckRank',
           14:'ScheduleRating',
           15:'ScheduleRatingRank',
           16:'OppO',
           17:'OppORank',
           18:'OppD',
           19:'OppDRank',
           20:'NCSOS',
           21:'NCSOSRank'}

int_cols = ['Rank',
           'AdjORank',
           'AdjDRank',
           'AdjTRank',
           'LuckRank',
           'ScheduleRatingRank',
           'OppORank',
           'OppDRank',
           'NCSOSRank',
           'Wins',
           'Losses']

float_cols = [
           'AdjEM',
           'AdjO',
           'AdjD',
           'AdjT',
           'Luck',
           'ScheduleRating',
           'OppO',
           'OppD',
           'NCSOS']

def create_pomeroy():
    df = pd.DataFrame()
    years = np.linspace(2002,2021,20).astype(int)
    col_rename = np.linspace(1,21,21)
    for year in years:
        #Scrape Table from site
        url = "https://kenpom.com/index.php?y="+str(year)
        yearly_df = pd.read_html(url)[0]
        
        #Fix feature names
        yearly_df.columns = col_rename
        yearly_df = yearly_df.rename(columns=mapping)
        
        #Cleaning extra title pickups
        yearly_df.dropna(inplace=True)
        yearly_df.drop(yearly_df[yearly_df['Rank'] == 'Rk'].index,inplace=True)
        
        #New columns
        yearly_df['Season'] = year
        yearly_df['TeamName'] = yearly_df['TeamName'].apply(lambda x: x[:-1] if x[-1] == '*' else x)
        yearly_df['TeamName'] = yearly_df['TeamName'].apply(lambda x: x[:-1] if x[-1] == '*' else x)
        yearly_df['TeamName'] = yearly_df['TeamName'].apply(lambda x: ' '.join(x.split()[:-1]) if x.split()[-1].isnumeric() else x)
        yearly_df['TeamName'] = yearly_df['TeamName'].apply(lambda x: x[:-1] if x[-1] == '.' else x)
        yearly_df['AdjEM'] = yearly_df['AdjEM'].apply(lambda x: x[1:] if x[0] == '+' else x)
        yearly_df['Luck'] = yearly_df['Luck'].apply(lambda x: x[1:] if x[0] == '+' else x)
        yearly_df['ScheduleRating'] = yearly_df['ScheduleRating'].apply(lambda x: x[1:] if x[0] == '+' else x)
        yearly_df['NCSOS'] = yearly_df['NCSOS'].apply(lambda x: x[1:] if x[0] == '+' else x)


        yearly_df[['Wins','Losses']] = yearly_df.Record.str.split("-",expand=True)
        for col in int_cols:
            yearly_df[col] = yearly_df[col].astype(int)
        
        for col in float_cols:
            yearly_df[col] = yearly_df[col].astype(float)
        
        #Combine to final table
        df = pd.concat([df,yearly_df],ignore_index=True)
    
    return df

In [8]:
df = create_pomeroy()

In [9]:
df[(df['TeamName'].str.contains('Dakota')) & (df['Season']==2009)]

,Rank,TeamName,Division,Record,AdjEM,AdjO,AdjORank,AdjD,AdjDRank,AdjT,...,ScheduleRatingRank,OppO,OppORank,OppD,OppDRank,NCSOS,NCSOSRank,Season,Wins,Losses
2381,61,North Dakota St,Sum,26-7,12.29,114.3,25,102.0,152,67.5,...,193,103.1,154,104.7,236,2.21,105,2009,26,7
2532,212,South Dakota St,Sum,13-20,-4.00,104.9,135,108.9,289,64.6,...,160,104.1,119,104.1,209,2.66,94,2009,13,20


In [11]:
team_ids = pd.read_csv(path+'MTeams.csv')

In [12]:
team_ids[team_ids['TeamID']==1315]

,TeamID,TeamName,FirstD1Season,LastD1Season
214,1315,North Dakota,2009,2021


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6867 entries, 0 to 6866
Data columns (total 24 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Rank                6867 non-null   int32  
 1   TeamName            6867 non-null   object 
 2   Division            6867 non-null   object 
 3   Record              6867 non-null   object 
 4   AdjEM               6867 non-null   float64
 5   AdjO                6867 non-null   float64
 6   AdjORank            6867 non-null   int32  
 7   AdjD                6867 non-null   float64
 8   AdjDRank            6867 non-null   int32  
 9   AdjT                6867 non-null   float64
 10  AdjTRank            6867 non-null   int32  
 11  Luck                6867 non-null   float64
 12  LuckRank            6867 non-null   int32  
 13  ScheduleRating      6867 non-null   float64
 14  ScheduleRatingRank  6867 non-null   int32  
 15  OppO                6867 non-null   float64
 16  OppORa

In [14]:
name_to_df_mapping = [('Mississippi Valley St','MS Valley St'),
                      ('Kent St','Kent'),
                      ('N.C. State','NC State'),
                      ('Western Kentucky','WKU'),
                      ('Southern Illinois','S Illinois'),
                      ("Saint Joseph's","St Joseph's PA"),
                      ("St. John's","St John's"),
                      ('St. Bonaventure','St Bonaventure'),
                      ('College of Charleston','Col Charleston'),
                      ('Saint Louis','St Louis'),
                      ('East Tennessee St','ETSU'),
                      ('Milwaukee','WI Milwaukee'),
                      ('Arkansas Little Rock','Ark Little Rock'),
                      ('Southwest Missouri St','Missouri St'),
                      ('Illinois Chicago','IL Chicago'),
                      ('Loyola Chicago','Loyola-Chicago'),
                      ('Western Michigan','W Michigan'),
                      ('Georgia Southern','Ga Southern'),
                      ('Louisiana Lafayette','Louisiana'),
                      ('Central Connecticut','Central Conn'),
                      ('The Citadel','Citadel'),
                      ('Troy St','Troy'),
                      ('Eastern Washington','E Washington'),
                      ('UTSA','UT San Antonio'),
                      ('UMKC','Missouri KC'),
                      ('Texas Pan American','UTRGV'),
                      ('UT Rio Grande Valley','UTRGV'),
                      ('Cal St. Northridge','CS Northridge'),
                      ('Northern Illinois','N Illinois'),
                      ('Louisiana Monroe','ULM'),
                      ('Florida Atlantic','FL Atlantic'),
                      ('Boston University','Boston Univ'),
                      ('George Washington','G Washington'),
                      ('Middle Tennessee','MTSU'),
                      ('Middle Tennessee St','MTSU'),
                      ('Central Michigan','C Michigan'),
                      ('American','American Univ'),
                      ('Green Bay','WI Green Bay'),
                      ("Saint Mary's","St Mary's CA"),
                      ('Southwest Texas St','Texas St'),
                      ('Monmouth','Monmouth NJ'),
                      ('Western Carolina','W Carolina'),
                      ('Eastern Illinois','E Illinois'),
                      ('FIU','Florida Intl'),
                      ('Northwestern St','Northwestern LA'),
                      ('Tennessee Martin','TN Martin'),
                      ('Texas A&M Corpus Chris','TAM C. Christi'),
                      ('St. Francis NY','St Francis NY'),
                      ('Western Illinois','W Illinois'),
                      ('Stephen F. Austin','SF Austin'),
                      ('Loyola Marymount','Loy Marymount'),
                      ('South Carolina St','S Carolina St'),
                      ('North Carolina A&T','NC A&T'),
                      ('Southeast Missouri St','SE Missouri St'),
                      ('Sacramento St','CS Sacramento'),
                      ('Charleston Southern','Charleston So'),
                      ('Cal St. Fullerton','CS Fullerton'),
                      ('Bethune Cookman','Bethune-Cookman'),
                      ('IPFW','PFW'),
                      ('Purdue Fort Wayne','PFW'),
                      ('Eastern Kentucky','E Kentucky'),
                      ('Maryland Eastern Shore','MD E Shore'),
                      ('Birmingham Southern','Birmingham So'),
                      ('Southeastern Louisiana','SE Louisiana'),
                      ('St. Francis PA','St Francis PA'),
                      ('Coastal Carolina','Coastal Car'),
                      ('Eastern Michigan','E Michigan'),
                      ("Saint Peter's","St Peter's"),
                      ('Grambling St','Grambling'),
                      ('Texas Southern','TX Southern'),
                      ('Albany','SUNY Albany'),
                      ('Southern','Southern Univ'),
                      ('Fairleigh Dickinson','F Dickinson'),
                      ('Prairie View A&M','Prairie View'),
                      ('Arkansas Pine Bluff','Ark Pine Bluff'),
                      ("Mount St. Mary's","Mt St Mary's"),
                      ('Utah Valley St','Utah Valley'),
                      ('Northern Colorado','N Colorado'),
                      ('North Dakota St','N Dakota St'),
                      ('South Dakota St','S Dakota St'),
                      ('Kennesaw St','Kennesaw'),
                      ('Central Arkansas','Cent Arkansas'),
                      ('Winston Salem St','W Salem St'),
                      ('USC Upstate','SC Upstate'),
                      ('Florida Gulf Coast','FL Gulf Coast'),
                      ('Cal St. Bakersfield','CS Bakersfield'),
                      ('North Carolina Central','NC Central'),
                      ('SIU Edwardsville','SIUE'),
                      ('Houston Baptist','Houston Bap'),
                      ('Nebraska Omaha','NE Omaha'),
                      ('Northern Kentucky','N Kentucky'),
                      ('UMass Lowell','MA Lowell'),
                      ('Abilene Christian','Abilene Chr'),
                      ('Fort Wayne','PFW'),
                      ('Little Rock','Ark Little Rock'),
                      ('Charleston','Col Charleston'),
                      ('LIU','LIU Brooklyn')]

In [15]:
for pair in name_to_df_mapping:
    df.loc[df['TeamName'] == pair[0], 'TeamName'] = pair[1]

In [16]:
df = df.merge(team_ids,on='TeamName',how='left')

In [17]:
features = ['Season','TeamID','Rank','AdjEM','AdjO','AdjD','Luck','ScheduleRating']

In [18]:
final_df = df[features]

In [19]:
final_df.to_csv('Pomeroy.csv',index=False)

In [20]:
final_df = pd.read_csv('Pomeroy.csv')

In [21]:
final_df[final_df['Season']==2021]

,Season,TeamID,Rank,AdjEM,AdjO,AdjD,Luck,ScheduleRating
6510,2021,1211,1,36.59,126.1,89.5,0.018,7.79
6511,2021,1276,2,30.70,119.5,88.8,0.026,16.78
6512,2021,1228,3,30.57,119.0,88.4,0.030,18.84
6513,2021,1124,4,29.57,123.4,93.8,0.050,9.65
6514,2021,1222,5,28.89,119.0,90.1,-0.009,6.65
...,...,...,...,...,...,...,...,...
6862,2021,1354,353,-24.94,87.1,112.0,-0.109,-4.63
6863,2021,1106,354,-26.19,84.2,110.4,-0.049,-15.12
6864,2021,1271,355,-27.17,84.0,111.2,0.000,-99.00
6865,2021,1152,356,-29.43,88.3,117.8,-0.006,9.05
